Test DMA by directly writing the CSRs

In [ ]:
import os
import json
from litex import RemoteClient

from functools import reduce

In [ ]:
soc_json = "../soc/streamliner.json"

In [ ]:
bus = RemoteClient()
bus.open()

In [ ]:
with open(soc_json) as file:
  soc = json.load(file)

In [ ]:
def set_base(addr):
    bus.write(soc['csr_registers']['wb_udp_tx_dma_base']['addr'], addr)

def read_base():
    return bus.read(soc['csr_registers']['wb_udp_tx_dma_base']['addr'])

def set_length(len):
    bus.write(soc['csr_registers']['wb_udp_tx_dma_length']['addr'], len)

def set_enable(dat):
    bus.write(soc['csr_registers']['wb_udp_tx_dma_enable']['addr'], dat)

def read_done():
    return bus.read(soc['csr_registers']['wb_udp_tx_dma_done']['addr'])

def set_loop(dat):
    bus.write(soc['csr_registers']['wb_udp_tx_dma_loop']['addr'], dat)

def set_port(src_port, dst_port):
    bus.write(soc['csr_registers']['wb_udp_tx_dma_srcdst_port']['addr'], (src_port<<16) + dst_port)

def str_ip4_to_num(x:str) -> int:
    return reduce(lambda x,y: x|y, map(lambda ix: (int(ix[1]) << (8 * ix[0])), enumerate(reversed(x.split(".")))))

def set_ip(ip:str):
    bus.write(soc['csr_registers']['wb_udp_tx_dma_dst_ip']['addr'], str_ip4_to_num(ip))

In [ ]:
set_enable(0)
set_base(0x40000000)
set_port(5123, 5123)
set_ip("192.168.100.20")
set_length(64)
set_loop(0)
set_enable(1)

In [ ]:
set_enable(0)

In [ ]:
read_done()

In [ ]:
bus.close()